# Chapter 2: End-to-End Machine Learning Project

**Book:** Hands-On Machine Learning with Scikit-Learn, Keras & TensorFlow (3rd Edition)

In [ ]:
# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Display settings
pd.options.display.max_columns = 20
pd.options.display.max_rows = 20
np.set_printoptions(precision=4, suppress=True)

# Plot settings
%matplotlib inline
plt.rc('font', size=12)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=12)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

## Working with Real Data

**Notes:**
- 

## Look at the Big Picture

**Notes:**
- 

## Get the Data

**Notes:**
- 

## Explore and Visualize the Data

**Notes:**
- 

## Prepare the Data for Machine Learning

**Notes:**
- 

## Select and Train a Model

**Notes:**
- 

## Fine-Tune Your Model

**Notes:**
- 

## Launch, Monitor, and Maintain

**Notes:**
- 

---
# Practical Examples

## Housing Price Prediction Project

In [ ]:
# Code for the housing project

## Exercises

1. 
2. 
3. 